In [1]:
# Importación de librerías para dataframes, rutas y manejo de fechas
import pandas as pd       # Para manipulación de dataframes
from pathlib import Path  # Para manejo de rutas de archivos
from datetime import datetime, timedelta  # Para manejo de fechas

In [2]:
R_Entrada = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Contributivo\Compensación\_Pila_SIE\relaciones laborales\Reporte_Afiliados Contributivo Relaciones Laborales_2025_04_07.csv"
R_Salida = r"\\Servernas\AYC2\ASEGURAMIENTO\ASEGURAMIENTO\PROCESO_ASEGURAMIENTO\REGIMEN SUBSIDIADO\MUNICIPIOS 2025\REPORTE RESOLUCION_0762_2023\04_Abril\02\SIE\ANEXOS\Maximas Relaciones Laborales.txt"

In [3]:
df_csv = pd.read_csv(R_Entrada, encoding='cp1252', sep=';', dtype=str)
print("Columnas:", df_csv.columns.tolist())
print("Cantidad de registros:", len(df_csv))

Columnas: ['tipo_documento', 'numero_identificacion', 'tipo_documento_aportante', 'numero_identificacion_aportante', 'razon_social', 'fecha_ingreso', 'Unnamed: 6']
Cantidad de registros: 24430


In [6]:
# Contar la cantidad de fechas distintas por combinación de "tipo_documento" y "numero_identificacion"
freq = df_csv.groupby(["tipo_documento", "numero_identificacion"])["fecha_ingreso"].nunique().reset_index(name="unique_fecha_count")

# Encontrar la combinación que aparece con más fechas distintas
max_combo = freq.sort_values("unique_fecha_count", ascending=False).iloc[0]

print("Combinación con más fechas distintas:")
print(max_combo)

# Filtrar el DataFrame para obtener los registros con esa combinación
df_max = df_csv[
    (df_csv["tipo_documento"] == max_combo["tipo_documento"]) &
    (df_csv["numero_identificacion"] == max_combo["numero_identificacion"])
]

print("\nRegistros con la combinación seleccionada:")
print(df_max)

Combinación con más fechas distintas:
tipo_documento                 CC
numero_identificacion    47435493
unique_fecha_count              6
Name: 14748, dtype: object

Registros con la combinación seleccionada:
      tipo_documento numero_identificacion tipo_documento_aportante  \
20645             CC              47435493                       CC   
20646             CC              47435493                       CC   
20647             CC              47435493                       CC   
20648             CC              47435493                       CC   
20649             CC              47435493                       CC   
20650             CC              47435493                       CC   

      numero_identificacion_aportante      razon_social fecha_ingreso  \
20645                       900218365  CONSULTORES INTE    2025-02-20   
20646                       900218365  CONSULTORES INTE    2025-02-10   
20647                       900218365  CONSULTORES INTE    2025-02-25   

In [7]:
# Número de registros antes de eliminar duplicados
registros_iniciales = df_csv.shape[0]
print("Registros antes:", registros_iniciales)

# Convertir la columna fecha_ingreso a tipo datetime
df_csv["fecha_ingreso"] = pd.to_datetime(df_csv["fecha_ingreso"], format='%Y-%m-%d')

# Mantener solo el registro más reciente por combinación única de tipo_documento y numero_identificacion
idx_max = df_csv.groupby(["tipo_documento", "numero_identificacion"])["fecha_ingreso"].idxmax()
df_csv_unico = df_csv.loc[idx_max].copy()

# Convertir la fecha al formato dd/mm/yyyy
df_csv_unico["fecha_ingreso"] = df_csv_unico["fecha_ingreso"].dt.strftime('%d/%m/%Y')

# Imprimir el número de registros luego de eliminar duplicados
registros_finales = df_csv_unico.shape[0]
print("Registros después:", registros_finales)

# Mostrar el dataframe resultante (opcional)
df_csv_unico

Registros antes: 24430
Registros después: 21435


,tipo_documento,numero_identificacion,tipo_documento_aportante,numero_identificacion_aportante,razon_social,fecha_ingreso,Unnamed: 6
0,CC,1000002981,CC,33645224,DIANA PATRICIA AFRICANO MORENO,02/01/2025,NaN
1,CC,1000049661,CC,1000049661,JUAN SEBASTIAN PIÑEROS GUTIERREZ,11/02/2025,NaN
2,CC,1000077323,CC,900154405,AVANZA COLOMBIA S A S,05/02/2025,NaN
3,CC,1000078116,CC,1118197333,CARLOS DAVID MURILLO ANACONA,29/03/2025,NaN
5,CC,1000122956,CC,800235997,REPRESENTACIONES Y DISTRIBUCIONES NOSOTROS SAS,12/10/2023,NaN
...,...,...,...,...,...,...,...
17424,TI,1222130285,TI,890900943,COLOMBIANA DE COMERCIO LTDA CORBETA,11/07/2022,NaN
17427,TI,1222130324,TI,47440204,RAQUEL BETSEY RUBIANAO BARRERA,14/03/2023,NaN
17428,TI,1222130354,TI,74751963,GAMEZ PARADA HEC,27/01/2025,NaN
17430,TI,1222130402,TI,860068255,COMPAÑIA DE SERVICIOS Y ADMINISTRACION S A SER...,27/01/2025,NaN


In [8]:
df_csv_unico.to_csv(R_Salida, sep=',', index=False)